url :

https://www.amentum.com/press-releases/

In [1]:
#libraries
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import pandas as pd,requests,bs4,re,time,io,pytesseract,easyocr,random,textstat,urllib.request
from pdfminer.high_level import extract_text
from PIL import Image
from pathlib import Path
from pdf2image import convert_from_path
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from goose3 import Goose
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.auto import tqdm

reader = easyocr.Reader(['en'])

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#driver for operation
from webdriver_manager.firefox import GeckoDriverManager
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())



====== WebDriver manager ======
Current firefox version is 101.0
Get LATEST geckodriver version for 101.0 firefox
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


In [3]:
SITE_NAME='amentum'

DOMAIN = "https://www.amentum.com"

SITE_LINK = "https://www.amentum.com/press-releases/"

In [4]:
#functions
def parse_webpage_bs(search_url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

def remove_esc_chars(string):
    review = re.sub('''[^a-zA-Z0-9!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]''', ' ',string)
    review = review.split()
    review = ' '.join(review)
    return review

def get_text_thumb(link): 
    try:
        driver.get(link)
        #most of the thumbnails we get after x path are invalid
        thumb = "https://www.amentum.com/wp-content/uploads/cropped-Amentum_RGB_large-2-e1615819496613-305x69-1.png"
        ps = driver.find_element(By.XPATH,f'''//*[@id="main"]/div/div/section[2]/div/div/div[1]/div/div/div[3]/div''').text.strip()
        text = remove_esc_chars(ps).strip()
    except:
        thumb = "https://www.amentum.com/wp-content/uploads/cropped-Amentum_RGB_large-2-e1615819496613-305x69-1.png"
        g = Goose()
        article_extract = g.extract(url=link)
        article = remove_esc_chars(article_extract.cleaned_text)
        meta_data = remove_esc_chars(article_extract.meta_description)
        whole_data = meta_data+article
        text = whole_data.strip()

        if textstat.lexicon_count(text, removepunct=True) < 5:
            try:
                response = requests.get(link)
                text = remove_esc_chars(extract_text(io.BytesIO(response.content)))

                if textstat.lexicon_count(text, removepunct=True) < 5:
                    texts = ""
                    r = requests.get(link)
                    filename = Path('temp.pdf')
                    filename.write_bytes(r.content)

                    pages = convert_from_path('temp.pdf', 500)
                    for x in pages:
                        x.save("temp.jpg")
                        output = reader.readtext("temp.jpg")
                        for o in output:
                            texts += o[1]

                    text = remove_esc_chars(texts)

                    if textstat.lexicon_count(text, removepunct=True) < 5:
                        texts = ""
                        soup = parse_webpage_bs(link)
                        if soup!= None:
                            ps = soup.findAll('p')
                            for p in ps:
                                texts+= p.text

                        text = remove_esc_chars(texts)
            except:
                text = ""
    
    return text , thumb

In [5]:
#get data
article_list = []

pagination = 1

#creating empty lists to append data 
published_dates,titles,texts,links,thumbnails,authors = [],[],[],[],[],[]

while True:
    url = f"https://www.amentum.com/press-releases/page/{pagination}"
    driver.get(url)
    
    elements = driver.find_elements(By.XPATH,'''//h3[@class="uael-post__title"]''')
    
    #breaking if there are no elements
    if len(elements) ==0:
        break
        
    #published_dates,titles,links,authors
    for e in tqdm(range(1,len(elements[:])+1),desc ="Getting urls"):
        pubished_date = driver.find_element(By.XPATH,f'''//*[@id="post-2318"]/div/div/div/div/section[2]/div/div/div[1]/div/div/div/div/div[2]/div/div/div[{e}]/div/div/div/div/span''').text.strip()
        published_dates.append(pubished_date)
        title = driver.find_element(By.XPATH,f'''//*[@id="post-2318"]/div/div/div/div/section[2]/div/div/div[1]/div/div/div/div/div[2]/div/div/div[{e}]/div/div/div/h3/a''').text.strip()
        titles.append(title)
        link = driver.find_element(By.XPATH,f'''//*[@id="post-2318"]/div/div/div/div/section[2]/div/div/div[1]/div/div/div/div/div[2]/div/div/div[{e}]/div/div/div/h3/a''').get_attribute('href')
        links.append(link)
        authors.append(SITE_NAME)
    
    pagination +=1 
    
#texts,thumbnails
for link in tqdm((links[:25]),desc ="Getting articles"):
    text , thumbnail = get_text_thumb(link)
    texts.append(text)
    thumbnails.append(thumbnail)

#zippig all the data togather 
zipped = list(zip(published_dates,titles,texts,links,thumbnails,authors))

#unwinding and appending to the main list
for published_date,title,text,link,thumbnail,author in zipped:
    article = (published_date.strip(),title.strip(),text.strip(),link.strip(),thumbnail.strip(),author.strip())
    article_list.append(article)
    print(published_date,title)

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

...

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

Getting urls:   0%|          | 0/6 [00:00<?, ?it/s]

Getting urls:   0%|          | 0/4 [00:00<?, ?it/s]

Getting articles:   0%|          | 0/25 [00:00<?, ?it/s]

June 7, 2022 Amentum Joint Venture Koa Lani Awarded Position on Remotely Piloted Aircraft IDIQ
May 24, 2022 Amentum’s United Cleanup Oak Ridge LLC Begins Critical Work at Oak Ridge Reservation
May 19, 2022 Amentum Awarded $48M Naval Test Wing Atlantic Extension
May 17, 2022 Amentum Awarded $99M Navy IDIQ to Support Air Crew Training in Nevada
May 10, 2022 Amentum Awarded $39M Army Contract for Research and Development
April 25, 2022 PAE JV PAE Korea Limited Wins War Reserve Materiel, Five-Year Contract Worth $45M
April 21, 2022 Amentum Achieves Certification for Occupational Health and Safety Management System by Developing Integrated Management System
April 12, 2022 Amentum Awarded $260M DHS Contract to Help Counter Unmanned System Threats
March 29, 2022 Amentum Announces New Leadership Team
March 21, 2022 Amentum Announces Hanford Leadership Changes
February 24, 2022 Amentum Announces John Heller as CEO
February 15, 2022 Amentum Completes Acquisition of PAE
February 8, 2022 Amentum A

In [6]:
temp_df = pd.DataFrame(article_list,columns=['date','title','article','url','thumbnail','author'])
temp_df

,date,title,article,url,thumbnail,author
0,"June 7, 2022",Amentum Joint Venture Koa Lani Awarded Positio...,"GERMANTOWN, Md.; June 7, 2022 Koa Lani, an Ame...",https://www.amentum.com/amentum-joint-venture-...,https://www.amentum.com/wp-content/uploads/cro...,amentum
1,"May 24, 2022",Amentum’s United Cleanup Oak Ridge LLC Begins ...,New UCOR Begins Operating Under New Contract G...,https://www.amentum.com/amentums-united-cleanu...,https://www.amentum.com/wp-content/uploads/cro...,amentum
2,"May 19, 2022",Amentum Awarded $48M Naval Test Wing Atlantic ...,"GERMANTOWN, Md.; May 19, 2022 The U.S. Navy aw...",https://www.amentum.com/amentum-awarded-48m-na...,https://www.amentum.com/wp-content/uploads/cro...,amentum
3,"May 17, 2022",Amentum Awarded $99M Navy IDIQ to Support Air ...,"GERMANTOWN, Md.; May 17, 2022 The Naval Air Wa...",https://www.amentum.com/amentum-awarded-99m-na...,https://www.amentum.com/wp-content/uploads/cro...,amentum
4,"May 10, 2022",Amentum Awarded $39M Army Contract for Researc...,"GERMANTOWN, Md.; May 10, 2022 Amentum has been...",https://www.amentum.com/amentum-awarded-39m-ar...,https://www.amentum.com/wp-content/uploads/cro...,amentum
5,"April 25, 2022",PAE JV PAE Korea Limited Wins War Reserve Mate...,"GERMANTOWN, Md., April 25, 2022 The 607 Materi...",https://www.amentum.com/pae-jv-partner-pae-kor...,https://www.amentum.com/wp-content/uploads/cro...,amentum
6,"April 21, 2022",Amentum Achieves Certification for Occupationa...,"GERMANTOWN, Md.; April 21, 2022 Amentum, a lea...",https://www.amentum.com/amentum-achieves-certi...,https://www.amentum.com/wp-content/uploads/cro...,amentum
7,"April 12, 2022",Amentum Awarded $260M DHS Contract to Help Cou...,"GERMANTOWN, Md.; April 12, 2022 Amentum has be...",https://www.amentum.com/amentum-awarded-260m-d...,https://www.amentum.com/wp-content/uploads/cro...,amentum
8,"March 29, 2022",Amentum Announces New Leadership Team,"GERMANTOWN, Md., March 29, 2022 Amentum, a lea...",https://www.amentum.com/amentum-announces-new-...,https://www.amentum.com/wp-content/uploads/cro...,amentum
9,"March 21, 2022",Amentum Announces Hanford Leadership Changes,"GERMANTOWN, Md., March 21, 2022 Scott Sax, pre...",https://www.amentum.com/amentum-announces-hanf...,https://www.amentum.com/wp-content/uploads/cro...,amentum


In [8]:
def see_data(iloc_no=random.randint(0,len(temp_df))-1):
    print(temp_df.iloc[iloc_no]['date'],temp_df.iloc[iloc_no]['title'])
    print(f"\n{temp_df.iloc[iloc_no]['author']} {temp_df.iloc[iloc_no]['url']}")
#     urllib.request.urlretrieve(temp_df.iloc[iloc_no]['thumbnail'], "temp.jpg")
#     display(Image.open("temp.jpg"))
    print(f"\n{temp_df.iloc[iloc_no]['article']}")

see_data()

November 30, 2021 Amentum Appoints Dr. Rob Smith to Lead Diversity, Equity and Inclusion

amentum https://www.amentum.com/amentum-appoints-dr-rob-smith-to-lead-diversity-equity-and-inclusion/

GERMANTOWN, Md.; Nov. 30, 2021 Amentum, a leading contractor to U.S. federal and allied governments, announced today that it has appointed Dr. Robert L. Smith as Vice President of Diversity, Equity & Inclusion. In this role, he will be responsible for leading Amentum s diversity, equity and inclusion vision and strategy by developing and implementing programs and initiatives that advance the company s business objectives and reinforces its culture. Rob brings with him deep expertise in developing and executing strategies that will help us live our values valuing each person in our organization, enabling them to reach their full potential and ultimately drive the success of our business, said John Vollmer, Amentum CEO. His leadership will be critical as we continue our journey to become the employ

In [9]:
#to csv
temp_df.to_csv(f'{SITE_NAME} news.csv',index = False)

#to json
temp_df.to_json(f'{SITE_NAME} news.json')

In [10]:
#to get rid of unwanteed trash created by the model use 
def remove_trash():
    try:
        try:
            os.remove("temp.pdf")
        except:
            pass
        os.remove("temp.jpg")
        print("Trash removed successfully")
    except:
        print("No trash found")

remove_trash()

No trash found


In [11]:
driver.quit()